In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import statsmodels.api as sm
import plotly.graph_objects as go

%matplotlib inline

#Reading the file

data = pd.read_csv(r'C:/Users/Dell/Desktop/Apollo563.csv')

#Creating Dummies

def get_dum(x):
    for elem in qs[x].unique():
        qs[str(elem)] = qs[x] == elem
        
qs = data[['uhid','Gender','MODE_OF_DELIVERY','inout_patient_status','Gestation','LOS','NEOFAX_ERRORS','MED_ADMIN_ERRORS','ESPGHAN','NUTR_ADMIN','RDS',
       'JAUNDICE', 'SEPSIS','baby_type','ANTENATA_STEROIDS']]

#Converting to Binary

def conv(x):
    if x == True:
        return 1
    else:
        return 0
    
get_dum('inout_patient_status')
qs.drop('inout_patient_status',axis=1,inplace=True)
get_dum('NEOFAX_ERRORS')
qs = qs.rename(columns={"No_err": "NF No_err", "ERROR": "NF ERROR"})
qs.drop('NEOFAX_ERRORS',axis=1,inplace=True)
get_dum('MED_ADMIN_ERRORS')
qs = qs.rename(columns={"No_err": "MED ADMIN No_err", "ERROR": "MED ADMIN ERROR", "Not eligible" : "MED ADMIN Not eligible"})
qs.drop('MED_ADMIN_ERRORS',axis=1,inplace=True)
get_dum('ESPGHAN')
qs = qs.rename(columns={"No_err": "ESPGHAN No_err", "ERROR": "ESPGHAN ERROR","Not eligible":"ESPGHAN NE"})
get_dum('NUTR_ADMIN')
qs = qs.rename(columns={"No_err": "NUTRI ADMIN No_err", "ERROR": "NUTRI ADMIN ERROR","Not eligible":"NUTRI ADMIN NE"})
get_dum('Gender')
get_dum('RDS')
qs = qs.rename(columns={"False": "RDS False", "True": "RDS True"})
get_dum('JAUNDICE')
qs = qs.rename(columns={"False": "Jaundice False", "True": "Jaundice True"})
get_dum('SEPSIS')
qs = qs.rename(columns={"False": "Sepsis False", "True": "Sepsis True"})
get_dum('MODE_OF_DELIVERY')
get_dum('baby_type')
get_dum('ANTENATA_STEROIDS')
qs = qs.rename(columns={"false": "Antenatal ST False", "true": "Antenatal ST True", "unknown":"Antenatal ST Unknown"})
qs.drop('MODE_OF_DELIVERY',axis=1,inplace=True)
qs.drop('baby_type',axis=1,inplace=True)
#qs.drop('nan',axis=1,inplace=True)
qs.drop('RDS',axis=1,inplace=True)
qs.drop('JAUNDICE',axis=1,inplace=True)
qs.drop('SEPSIS',axis=1,inplace=True)
qs.drop('NUTR_ADMIN',axis=1,inplace=True)
qs.drop('ESPGHAN',axis=1,inplace=True)
qs.drop('Gender',axis=1,inplace=True)
qs.drop('ANTENATA_STEROIDS',axis=1,inplace=True)

l = ['In Born', 'Out Born', 'NF No_err', 'NF ERROR',
        'MED ADMIN No_err', 'MED ADMIN ERROR',
       'ESPGHAN ERROR', 'ESPGHAN No_err', 'NUTRI ADMIN ERROR',
       'NUTRI ADMIN No_err','NUTRI ADMIN No_err' ,'Male', 'Female', 'RDS False', 'RDS True',
       'Jaundice False', 'Jaundice True', 'Sepsis False', 'Sepsis True',
       'LSCS', 'NVD', 'Single', 'Multiple', 'False',
       'True']

for i in l:
    qs[i] = qs[i].apply(conv)
    
#Model is defined
    
def model(data):
    y = data['LOS']
    X = data.drop('LOS',axis=1)
    X.drop('uhid',axis=1,inplace=True)
    NegBi_model = sm.GLM(y,X, family=sm.families.NegativeBinomial())
    NegBi_results = NegBi_model.fit()
    results_as_html = NegBi_results.summary().tables[1].as_html()
    dt = pd.read_html(results_as_html, header=0, index_col=0)[0]

    return (NegBi_results.params, dt)

data_34 = qs[qs['Gestation']<34]
data_36 = qs[(qs['Gestation']>=34) & (qs['Gestation']<=37)]
data_37 = qs[qs['Gestation']>37]

#Reference Grid for LS Means

def refgrid(x):

    gest = [x[0]]
    inout = [x[2],x[1]]
    NF = [x[3],x[4]]
    med_admin = [x[6],x[5]]
    espghan = [x[7],x[8]]
    nutri = [x[9],x[10]]
    gender = [x[11],x[12]]
    rds = [x[14],x[13]]
    jaundice = [x[15],x[16]]
    sepsis = [x[17],x[18]]
    typelsnv = [x[19],x[20]]
    type123 = [x[21],x[22]]
    anst = [x[23],x[24]]
    los_combo = list(itertools.product(inout,NF,med_admin,espghan,gender,nutri,rds,jaundice,sepsis,typelsnv,type123,anst,gest))
    los_df = pd.DataFrame(los_combo)
    los_df = los_df.rename(columns={0: "inout", 1: "NF", 2 : "Med_admin", 3: "espghan", 4:"nutri", 5:"gender", 6:"rds", 7:"jaundice", 8:"sepsis",9:"lscs_nvd",10:"birth_type",11:"ant_std",12:"Gestation"})
    return los_df

#Calculation for each gestation group

def choice34_37(x):
    z, m = model(data_36)
    
    d = []
    e = []
    a = refgrid(z)
    q ={"inout":1, "NF":3, "Med_admin":5, "espghan":7, "nutri":9, "gender":11, "rds":13, "jaundice":15, "sepsis":17, "lscs_nvd":19, "birth_type":21,"ant_std":23}
    p = q[x]
    
    los = a
    los_new = a.drop(str(x),axis=1)
    los_1 = los_new.drop('Gestation',axis=1)
    los_1['sum'] = los_new.sum(axis=1)
    
    
    
    
    if m['P>|z|'][p] < .05 and m['P>|z|'][p+1] < .05:
           
        #los1 = qs[qs['Gestation']<34]
        #median1 = np.median(los1['LOS'])
        
        for j in range(len(los_new)):
            for i in np.arange(34.0,37.0,0.1):
                d.append(los_1['sum'].iloc[j] + (los_new['Gestation'].iloc[j])*i + los[x].iloc[0])
                e.append(los_1['sum'].iloc[j] + (los_new['Gestation'].iloc[j])*i + los[x].iloc[4095])
        s = pd.DataFrame(d)
        t = pd.DataFrame(e)
        
        return(round(np.exp(s.mean())[0]-data_36.median()[1]),round(np.exp(t.mean())[0]-data_36.median()[1]))
    elif m['P>|z|'][p] < .05 and m['P>|z|'][p+1] > .05:
        for j in range(len(los_new)):
            for i in np.arange(34.0,37.0,0.1):
                d.append(los_1['sum'].iloc[j] + (los_new['Gestation'].iloc[j])*i + los[x].iloc[0])
                e.append(0)
        s = pd.DataFrame(d)
        t = pd.DataFrame(e) 
        
        return(round(np.exp(s.mean())[0]-data_36.median()[1]),' ')
    elif m['P>|z|'][p] > .05 and m['P>|z|'][p+1] < .05:
        for j in range(len(los_new)):
            for i in np.arange(34.0,37.0,0.1):
                e.append(los_1['sum'].iloc[j] + (los_new['Gestation'].iloc[j])*i + los[x].iloc[4095])
                d.append(0)
        s = pd.DataFrame(d)
        t = pd.DataFrame(e) 
        
        return(' ',round(np.exp(t.mean())[0]))
                
        
        
        
    else:
        return (' ', ' ')
    
def choice25_34(x):
    z, m = model(data_34)
    
    d = []
    e = []
    a = refgrid(z)
    q ={"inout":1, "NF":3, "Med_admin":5, "espghan":7, "nutri":9, "gender":11, "rds":13, "jaundice":15, "sepsis":17, "lscs_nvd":19, "birth_type":21,"ant_std":23}
    p = q[x]
    
    los = a.copy()
    los_new = a.drop(str(x),axis=1)
    los_1 = los_new.drop('Gestation',axis=1)
    los_1['sum'] = los_new.sum(axis=1)
    
    
    
    
    if m['P>|z|'][p] < .05 and m['P>|z|'][p+1] < .05:
           
        #los1 = qs[qs['Gestation']<34]
        #median1 = np.median(los1['LOS'])
        
        for j in range(len(los_new)):
            for i in np.arange(25.0,34.0,0.1):
                d.append(los_1['sum'].iloc[j] + (los_new['Gestation'].iloc[j])*i + los[x].iloc[0])
                e.append(los_1['sum'].iloc[j] + (los_new['Gestation'].iloc[j])*i + los[x].iloc[4095])
        s = pd.DataFrame(d)
        t = pd.DataFrame(e)
        
        
        
        return(round(np.exp(s.mean())[0]-data_34.median()[1]),round(np.exp(t.mean())[0]-data_34.median()[1]))
    elif m['P>|z|'][p] < .05 and m['P>|z|'][p+1] > .05:
        for j in range(len(los_new)):
            for i in np.arange(25.0,34.0,0.1):
                d.append(los_1['sum'].iloc[j] + (los_new['Gestation'].iloc[j])*i + los[x].iloc[0])
                e.append(0)
        s = pd.DataFrame(d)
        t = pd.DataFrame(e) 
        
        return(round(np.exp(s.mean())[0]-data_34.median()[1]),' ')
    elif m['P>|z|'][p] > .05 and m['P>|z|'][p+1] < .05:
        for j in range(len(los_new)):
            for i in np.arange(25.0,34.0,0.1):
                e.append(los_1['sum'].iloc[j] + (los_new['Gestation'].iloc[j])*i + los[x].iloc[4095])
                d.append(0)
        s = pd.DataFrame(d)
        t = pd.DataFrame(e) 
        
        return(' ',round(np.exp(t.mean())[0]))
                
        
        
        
    else:
        return (' ', ' ')

def choice37_44(x):
    z, m = model(data_37)
    
    d = []
    e = []
    a = refgrid(z)
    q ={"inout":1, "NF":3, "Med_admin":5, "espghan":7, "nutri":9, "gender":11, "rds":13, "jaundice":15, "sepsis":17, "lscs_nvd":19, "birth_type":21,"ant_std":23}
    p = q[x]
    
    los = a
    los_new = a.drop(str(x),axis=1)
    los_1 = los_new.drop('Gestation',axis=1)
    los_1['sum'] = los_new.sum(axis=1)
    
    
    
    
    if m['P>|z|'][p] < .05 and m['P>|z|'][p+1] < .05:
           
        #los1 = qs[qs['Gestation']<34]
        #median1 = np.median(los1['LOS'])
        
        for j in range(len(los_new)):
            for i in np.arange(37.0,44.0,0.1):
                d.append(los_1['sum'].iloc[j] + (los_new['Gestation'].iloc[j])*i + los[x].iloc[0])
                e.append(los_1['sum'].iloc[j] + (los_new['Gestation'].iloc[j])*i + los[x].iloc[4095])
        s = pd.DataFrame(d)
        t = pd.DataFrame(e)
        
        
        return(round(np.exp(s.mean())[0]-data_37.median()[1]),round(np.exp(t.mean())[0]-data_37.median()[1]))
    elif m['P>|z|'][p] < .05 and m['P>|z|'][p+1] > .05:
        for j in range(len(los_new)):
            for i in np.arange(37.0,44.0,0.1):
                d.append(los_1['sum'].iloc[j] + (los_new['Gestation'].iloc[j])*i + los[x].iloc[0])
                e.append(0)
        s = pd.DataFrame(d)
        t = pd.DataFrame(e) 
        
        return(round(np.exp(s.mean())[0]-data_37.median()[1]),' ')
    elif m['P>|z|'][p] > .05 and m['P>|z|'][p+1] < .05:
        for j in range(len(los_new)):
            for i in np.arange(37.0,44.0,0.1):
                e.append(los_1['sum'].iloc[j] + (los_new['Gestation'].iloc[j])*i + los[x].iloc[4095])
                d.append(0)
        s = pd.DataFrame(d)
        t = pd.DataFrame(e) 
        
        return(' ',round(np.exp(t.mean())[0]))
                
        
        
        
    else:
        return (' ', ' ')
    
#Median IQR
    
def median_IQR(x):
    q1 = str(pd.DataFrame(x).quantile(.25)[0])
    q2 = str(pd.DataFrame(x).quantile(.75)[0])
    q3 = q1 +" to " + q2
    return q3

Dict1 = {'Less than 34' : {'Count': str(data_34.count()[0]), 'Median': str(data_34.median()[1]),'IQR': median_IQR(data_34['LOS']),'Male':choice25_34('gender')[0],'Female' : choice25_34('gender')[1],'Out Born' : choice25_34('inout')[0],'In Born' : choice25_34('inout')[1], 'LSCS' : choice25_34('lscs_nvd')[0],'NVD' : choice25_34('lscs_nvd')[1],'Single' : choice25_34('birth_type')[0], 'Multiple' : choice25_34('birth_type')[1],'Antenatal Steroids False' : choice25_34('ant_std')[1],'Antenatal Steroids True' : choice25_34('ant_std')[0],'NeoFax No Error' : choice25_34('NF')[0],'NeoFax Error' : choice25_34('NF')[1],'Med Admin Error' : choice25_34('Med_admin')[1],'Med Admin No Error' : choice25_34('Med_admin')[0],'ESPGHAN Error' : choice25_34('espghan')[0],'ESPGHAN No Error' : choice25_34('espghan')[1],'Nutri Admin No Error' : choice25_34('nutri')[1],'Nutri Admin Error' : choice25_34('nutri')[0],'RDS True': choice25_34('rds')[0],'RDS False':choice25_34('rds')[1],'Jaundice False':choice25_34('jaundice')[0],'Jaundice True':choice25_34('jaundice')[1],'Sepsis True':choice25_34('sepsis')[0],'Sepsis False':choice25_34('sepsis')[1]},
        '34 to 37' : {'Count': str(data_36.count()[0]), 'Median': str(data_36.median()[1]),'IQR': median_IQR(data_36['LOS']),'Male':choice34_37('gender')[0],'Female' : choice34_37('gender')[1],'Out Born' : choice34_37('inout')[0],'In Born' : choice34_37('inout')[1], 'LSCS' : choice34_37('lscs_nvd')[0],'NVD' : choice34_37('lscs_nvd')[1],'Single' : choice34_37('birth_type')[0], 'Multiple' : choice34_37('birth_type')[1],'Antenatal Steroids False' : choice34_37('ant_std')[1],'Antenatal Steroids True' : choice34_37('ant_std')[0],'NeoFax No Error' : choice34_37('NF')[0],'NeoFax Error' : choice34_37('NF')[1],'Med Admin Error' : choice34_37('Med_admin')[1],'Med Admin No Error' : choice34_37('Med_admin')[0],'ESPGHAN Error' : choice34_37('espghan')[0],'ESPGHAN No Error' : choice34_37('espghan')[1],'Nutri Admin No Error' : choice34_37('nutri')[1],'Nutri Admin Error' : choice34_37('nutri')[0],'RDS True': choice34_37('rds')[0],'RDS False':choice34_37('rds')[1],'Jaundice False':choice34_37('jaundice')[0],'Jaundice True':choice34_37('jaundice')[1],'Sepsis True':choice34_37('sepsis')[0],'Sepsis False':choice34_37('sepsis')[1]},
        'Greater than 37' : {'Count': str(data_37.count()[0]), 'Median': str(data_37.median()[1]),'IQR': median_IQR(data_37['LOS']),'Male':choice37_44('gender')[0],'Female' : choice37_44('gender')[1],'Out Born' : choice37_44('inout')[0],'In Born' : choice37_44('inout')[1], 'LSCS' : choice37_44('lscs_nvd')[0],'NVD' : choice37_44('lscs_nvd')[1],'Single' : choice37_44('birth_type')[0], 'Multiple' : choice37_44('birth_type')[1],'Antenatal Steroids False' : choice37_44('ant_std')[1],'Antenatal Steroids True' : choice37_44('ant_std')[0],'NeoFax No Error' : choice37_44('NF')[0],'NeoFax Error' : choice37_44('NF')[1],'Med Admin Error' : choice37_44('Med_admin')[1],'Med Admin No Error' : choice37_44('Med_admin')[0],'ESPGHAN Error' : choice37_44('espghan')[0],'ESPGHAN No Error' : choice37_44('espghan')[1],'Nutri Admin No Error' : choice37_44('nutri')[1],'Nutri Admin Error' : choice37_44('nutri')[0],'RDS True': choice37_44('rds')[0],'RDS False':choice37_44('rds')[1],'Jaundice False':choice37_44('jaundice')[0],'Jaundice True':choice37_44('jaundice')[1],'Sepsis True':choice37_44('sepsis')[0],'Sepsis False':choice37_44('sepsis')[1]}}
        
headerColor = 'grey'
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>FIELDS</b>','<b>Less than 34</b>','<b>34 to 37</b>','<b>Greater than 37</b>'],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      ['Count','Median','IQR','Male','Female', 'Out born','In Born', 'LSCS','NVD','Single' ,'Multiple', 'Antenatal Steroids False','Antenatal Steroids True','NeoFax Error','NeoFax No Error','Med Admin Error','Med Admin No Error','ESPHGAN Error','ESPGHAN No Error','Nutri Admin Error','Nutri Admin No Error','RDS True','RDS False','Jaundice True','Jaundice False','Sepsis True','Sepsis False'],
      [data_34.count()[0],data_34.median()[1],median_IQR(data_34['LOS']),choice25_34('gender')[0],choice25_34('gender')[1], choice25_34('inout')[0],choice25_34('inout')[1], choice25_34('lscs_nvd')[0],choice25_34('lscs_nvd')[1], choice25_34('birth_type')[0],choice25_34('birth_type')[1],choice25_34('ant_std')[1],choice25_34('ant_std')[0], choice25_34('NF')[0],choice25_34('NF')[1],choice25_34('Med_admin')[1],choice25_34('Med_admin')[0],choice25_34('espghan')[0],choice25_34('espghan')[1],choice25_34('nutri')[1],choice25_34('nutri')[0],choice25_34('rds')[0],choice25_34('rds')[1],choice25_34('jaundice')[0],choice25_34('jaundice')[1],choice25_34('sepsis')[0],choice25_34('sepsis')[1]],
      [data_36.count()[0],data_36.median()[1],median_IQR(data_36['LOS']),choice34_37('gender')[0],choice34_37('gender')[1], choice34_37('inout')[0],choice25_34('inout')[1], choice34_37('lscs_nvd')[0],choice34_37('lscs_nvd')[1], choice34_37('birth_type')[0],choice34_37('birth_type')[1],choice34_37('ant_std')[1],choice34_37('ant_std')[0], choice34_37('NF')[0],choice34_37('NF')[1],choice34_37('Med_admin')[1],choice34_37('Med_admin')[0],choice34_37('espghan')[0],choice34_37('espghan')[1],choice34_37('nutri')[1],choice34_37('nutri')[0],choice34_37('rds')[0],choice34_37('rds')[1],choice34_37('jaundice')[0],choice34_37('jaundice')[1],choice34_37('sepsis')[0],choice34_37('sepsis')[1]],
      [data_37.count()[0],data_37.median()[1],median_IQR(data_37['LOS']),choice37_44('gender')[0],choice37_44('gender')[1], choice37_44('inout')[0],choice25_34('inout')[1], choice37_44('lscs_nvd')[0],choice37_44('lscs_nvd')[1], choice37_44('birth_type')[0],choice37_44('birth_type')[1],choice37_44('ant_std')[1],choice37_44('ant_std')[0], choice37_44('NF')[0],choice37_44('NF')[1],choice37_44('Med_admin')[1],choice37_44('Med_admin')[0],choice37_44('espghan')[0],choice37_44('espghan')[1],choice37_44('nutri')[1],choice37_44('nutri')[0],choice37_44('rds')[0],choice37_44('rds')[1],choice37_44('jaundice')[0],choice37_44('jaundice')[1],choice37_44('sepsis')[0],choice37_44('sepsis')[1]],

    ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.show()        
